# Inventário

Deve-se executar as seis primeiras células para carregar funções utilizadas para rodar o script

In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from simple_history.utils import bulk_create_with_history
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from helpers.histories import bulk_update_with_history
import pytz

In [2]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [3]:
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from django.contrib.gis.db.models.functions import Distance
from turfpy.misc import nearest_point_on_line
from turfpy.measurement import length
from geojson import LineString, loads
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [20]:
def point_to_km(lng, lat, road):
    point = Point(lng, lat, srid=4326)
    if isinstance(road,str):
        roads = Road.objects.filter(company=company,name__icontains=road).annotate(distance=Distance(point, 'path')).order_by('distance')
        road = roads[0]
    else:
        road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [9]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [10]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [11]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [12]:
def get_sign_color(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.capitalize().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [13]:
def get_sign_film(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.upper().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

 Verificando se está no ambiente de produção

In [14]:
!cat .env

cat: .env: No such file or directory


Escolhendo nome do arquivo Excel para carregar o Inventário

In [118]:
filename='Bueiro-BRS-470 OK REV1'
road_name='470'

Escolhendo a aba da planilha do Excel

In [119]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
ws = wb[sheetname]

Testando planilha carregada do Excel

In [120]:
print(ws['A1'].value)

km


Carregando os valores no script

In [121]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    if obj['Rodovia'] is not None:           
   
        obj['img'] = [a for a in ws._images if a.anchor._from.row == index]
#         for i,img in enumerate(obj['img']):
#             obj['img'][i].name = ws.cell(row=img.anchor._from.row, column=col_index+3).value if ws.cell(row=img.anchor._from.row, column=col_index+3).value is not None else 'CSG'
            
        if len(obj['img']) >=3:
            try:
                obj['img'][0].name='Direita'
                obj['img'][1].name='Central'
                obj['img'][2].name='Esquerda'
            except:
                pass
        elif len(obj['img']) == 2:
            try:
                obj['img'][0].name='Direita'
                obj['img'][1].name='Esquerda'
            except:
                pass
        else:
            try:
                obj['img'][0].name='Direita'
            except:
                pass
        
    values.append(obj)

Definindo valores da importação

In [122]:
company = Company.objects.get(name='Caminhos da Serra Gaúcha')

#Classe Bueiros
occurrence_type=OccurrenceType.objects.get(uuid='083d75c7-0bf7-4a0b-bdb8-4f43f10f9266')

user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Identificado')
# step=ApprovalStep.objects.get(approval_flow__company=company, name='Em Elaboração')
firm=Firm.objects.get(uuid='0edd2ecc-8d5e-4444-898b-0069a3e9c354')
company,occurrence_type,firm

(<Company: 4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha>,
 <OccurrenceType: Bueiro - ['Caminhos da Serra Gaúcha']>,
 <Firm: [4edb7778-e350-4e77-8e1e-de5f87b1da7f: Caminhos da Serra Gaúcha] 0edd2ecc-8d5e-4444-898b-0069a3e9c354: Cadastro incial>)

In [123]:
# road=Road.objects.get(company=company,name__icontains="470")
# items=list(road.marks.items())
# first_km=items[0][1]['km']
# last_km=items[-1][1]['km']
# print(first_km,last_km)

In [124]:
for a in values:
    a['Latitude_Inicial'] = '-'+str(a['Latitude_Inicial']) if '-' not in str(a['Latitude_Inicial']) else str(a['Latitude_Inicial'])
    a['Longitude_Inicial'] = '-'+str(a['Longitude_Inicial']) if '-' not in str(a['Longitude_Inicial']) else str(a['Longitude_Inicial'])
    if a['Lado montante'] == 'Central':
        a['Lado montante'] = "Centro"
    if a['Lado montante'] == 'Esquedo':
        a['Lado montante'] = "Esquerdo"

In [125]:
len(values),values[0]

(198,
 {'km': None,
  'km final': None,
  'km de Projeto': None,
  'km final de Projeto': None,
  'Latitude_Inicial': '-30.57611111111111',
  'Longitude_Inicial': '-51.714166666666664',
  'Latitude_Final': None,
  'Longitude_Final': None,
  'Status': 'Identificado',
  'Equipe/Empreiteira': 'Cadastro incial',
  'Encontrado em': None,
  'Executado em': None,
  'Sentido': 'Não se Aplica',
  'Classe': 'Bueiro',
  'Faixa': 'Não se aplica',
  'Lado montante': 'Direito',
  'Rodovia': 'BRS-470',
  'Tipo': 'BSTC',
  'Dimensão\n(m)': 0.8,
  'Extensão\n(m)': 118.69330225418172,
  'Dispositivos de Entrada': 'BOCA',
  'Dispositivos de Saída': 'BOCA',
  'Estado de Conservação': 'Regular',
  'Observações': None,
  'Foto_1': None,
  'Data Foto_1': None,
  'Tipo Foto_1': None,
  'Descrição Foto_1': 'Direita',
  'Foto_2': None,
  'Data Foto_2': 'Esquerda',
  'Tipo Foto_2': None,
  'Foto_3': None,
  'Data Foto_3': None,
  'Tipo Foto_3': None,
  'Descrição Foto_3': None,
  'Foto_4': None,
  'Data Foto_4':

Criando itens de Inventário

In [126]:
objects=[]


for index, a in enumerate(tqdm(values)):
#     print(float(a['Longitude_Inicial']),float(a['Latitude_Inicial']))
    road,_,point,km =point_to_km(float(a['Longitude_Inicial']),float(a['Latitude_Inicial']),road_name)
    if a['Longitude_Final'] is not None:
        _,_,_,end_km =point_to_km(float(a['Longitude_Final']),float(a['Latitude_Final']),road_name)
    if road_name not in road.name:
        print(f"Rodovia {road} - Km:{km} - Linha: {index+2}")
    try:

        objects.append((Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                    road=road,
                    road_name=road.name,
                    direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                    created_by=user,
#                     found_at=datetime.datetime('2023-05-08'),
                    km=float(km),
                    end_km_manually_specified=True if a['Longitude_Final'] is not None else False,
                    end_km=float(end_km) if a['Longitude_Final'] is not None else None,
                    point=point,
                    firm=firm,
                    status=status,
                    form_data={
                        'type': a['Tipo'],
                        'side': get_value('Lado montante', occurrence_type, a['Lado montante']) if a['Lado montante'] is not None else None,
#                         'side': a['Lado montante'].strftime('%Y-%m-%dT03:00:00.000Z') if a['Data de fabricação'] and type(a['Data de fabricação']) is datetime.datetime else None,
                        'state': get_value('Estado de conservação', occurrence_type, a['Estado de conservação'].capitalize() if a.get('Estado de conservação') else a['Estado de Conservação'].capitalize()) if a.get('Estado de conservação') is not None or a.get('Estado de Conservação') is not None else None,
                        'dimensao': str(a['Dimensão\n(m)']),
                        'extension': a['Extensão\n(m)'],
                        'dispentrada': a['Dispositivos de Entrada'],
                        'dispsaida': a['Dispositivos de Saída'],                                                     
                    }, 
                    ),a['img']))
    except Exception as e:

        print(f'Arquivo"{filename}" - Rodovia {road} - Km:{km} - Linha: {index+2} - {e}')
        print(objects.form_data)

  0%|          | 0/198 [00:00<?, ?it/s]

ValueError: could not convert string to float: '-None'

Salvando itens de Inventário no sistema e imprimindo os seriais

In [134]:
for reporting, images in tqdm(objects):
    print(reporting.point[0],' | ',reporting.point[1],' | ',reporting.km)

  0%|          | 0/69 [00:00<?, ?it/s]

-51.714166666666664  |  -30.57611111111111  |  233.387
-51.72277777777778  |  -29.243333333333332  |  225.619
-51.64972222222222  |  -29.593888888888888  |  233.387
-51.62  |  -29.70861111111111  |  233.387
-51.63361111111111  |  -29.988333333333333  |  233.387
-51.64361111111111  |  -30.04277777777778  |  233.387
-51.658055555555556  |  -30.076666666666668  |  233.387
-51.63361111111111  |  -30.24111111111111  |  233.387
-51.67027777777778  |  -30.52166666666667  |  233.387
-51.61555555555555  |  -30.775  |  233.387
-51.61555555555555  |  -30.775  |  233.387
-51.611666666666665  |  -30.82027777777778  |  233.387
-51.59888888888889  |  -29.226666666666667  |  225.568
-51.5925  |  -29.260833333333334  |  225.676
-51.58583333333333  |  -29.30361111111111  |  227.567
-51.577222222222225  |  -29.34777777777778  |  232.949
-51.57  |  -29.389166666666668  |  233.028
-51.556666666666665  |  -29.43888888888889  |  233.387
-51.54083333333333  |  -29.52111111111111  |  233.387
-51.53472222222222

In [115]:
for reporting, images in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/42 [00:00<?, ?it/s]

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28887
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28888
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28889
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28890
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28891
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28892
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28893
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CSG-Inv-2023.28894
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2044: RSC-287
CS

Anexando as imagens

In [116]:
for reporting, images in tqdm(objects):
    for image in images:
        if hasattr(image, 'name'):
#             print('O objeto tem o atributo nome!')
            pass
        else:
#             print('O objeto não tem o atributo nome.')
            image.name="CSG"
            print(image.name)

  0%|          | 0/42 [00:00<?, ?it/s]

In [117]:
for reporting, images in tqdm(objects):
    for image in images:
        reporting_file = ReportingFile(
            created_by=user,
            reporting=reporting,
            description=image.name,
            km=reporting.km,
            point=reporting.point
        )
        reporting_file.save()
        image_io = BytesIO(image._data())# Cria um espaço na memória em formato binário, com o conteúdo da imagem
        im = Image.open(image_io) # Abre a imagem criada acima com a biblioteca PIL
        if im.mode in ("RGBA", "P"):
            im = im.convert("RGB") # Converte a imagem para formatos aceitos do sistema, caso tenha vindo fora do padrão
        thumb_io = BytesIO() # Cria uma espaço outro espaço na memória para salvar a imagem
        im.save(thumb_io, format='jpeg', quality=90) # Salva a imagem no espaço criado acima
        reporting_file.upload.save(image.name + '.jpeg', thumb_io) # Salva o conteúdo da imagem no S3

  0%|          | 0/42 [00:00<?, ?it/s]

In [23]:
reps = Reporting.objects.filter(company=company,occurrence_type=occurrence_type,created_at__gte='2023-5-10')
reps.count()

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.created_at received a naive datetime (2023-05-10 00:00:00) while time zone support is active.
  RuntimeWarning)


169

In [24]:
for a in tqdm(reps):
    a.delete()

  0%|          | 0/169 [00:00<?, ?it/s]